# Zadanie 1
Aproksymacja funkcji sin i kwadratowej, z wykorzystaniem Keras i Tensorflow.

### Potrzebne moduły

In [1]:
import tensorflow as tf
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, clear_output

Using TensorFlow backend.


In [7]:
np.random.seed(19)
tf.random.set_seed(19)

### Klasa rozszerzająca wbudowany moduł Callbacks
Pozwala na dopisanie własnej funckjonalności. W tym wypadku będzie to zapisywanie przewidywań sieci co setny epoch, aby potem przedstawić te dane na wykresie.

In [3]:
class CustomCallback(Callback):
    def __init__(self, data, visible):
        super().__init__()
        self.predict_data_for_NN = data
        self.predict_data_for_plot = [float(i[0]) for i in data]
        self.visible = visible
        self.history = []

    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch, logs)
        if (epoch+1) % 100 == 0:
            predict_Y = [i[0] for i in self.model.predict(self.predict_data_for_NN)]
            self.history.append(predict_Y)
            if self.visible:
                clear_output(wait=True)
                print(f'Callecting data {epoch+1} iteration')

    def getHistory(self):
        return self.history

### Klasa odpowiadająca za animację
Pozwala stworzyć dwa wykresy - aproksymowanej funkcji oraz rzeczywiestycz zmian, oraz animować aktualizacje stanów.

In [4]:
class Anim(object):
    def __init__(self, history, minx, maxx, miny, maxy, control_X, control_Y):
        self.history = history
        self.minx = minx
        self.maxx = maxx
        self.fig, (ax0, self.ax1) = plt.subplots(nrows=2, figsize=(12, 6))
        plt.subplots_adjust(hspace=0.5)
        ax0.set_xlim((minx, maxx))
        ax0.set_ylim((miny, maxy))
        ax0.set_title('Oryginalny wykres')
        self.ax1.set_xlim((minx, maxx))
        self.ax1.set_ylim((miny, maxy))
        self.ax1.set_title('Aproksymacja przez sieć neuronową')
        self.ax1.set_xlabel('Iteration 0')
        self.line1, = ax0.plot(control_X, control_Y)
        self.line2, = self.ax1.plot([], [])
    
    def getFig(self):
        return self.fig
    
    def actualize(self, i):
        x = np.linspace(self.minx, self.maxx, 101)
        y = self.history[i]
        self.line2.set_data(x, y)
        self.ax1.set_xlabel(f'Iteration {i*100}')
        return (self.line2,)

### Funkcja tworząca sieć

In [5]:
def runNetwork(X, Y, act, los, opt, visible):
    model = Sequential()
    model.add(Dense(10, activation=act, kernel_initializer='he_uniform'))
    model.add(Dense(10, activation=act, kernel_initializer='he_uniform'))
    model.add(Dense(1))

    model.compile(loss=los, optimizer=opt)
    
    callback = CustomCallback(X, visible)
    model.fit(X, Y, epochs=5000, batch_size=10, verbose=0, callbacks=[callback])
    return callback.getHistory(), model

# Animacja uczenia z wybranymi parametrami

### Funkcja kwadratowa

In [8]:
# %matplotlib inline

control_X = np.linspace(-50, 50, 26)
control_Y = control_X ** 2

X = np.array([[float(i)] for i in np.linspace(-50, 50, 101)])
Y = np.array([[float(i ** 2)] for i in np.linspace(-50, 50, 101)])

hist, model = runNetwork(X, Y, 'relu', 'mean_squared_error', 'adam', visible=True)
print('Creating animation ...')
anim = Anim(hist, -50, 50, -100, 2600, control_X, control_Y)
a = animation.FuncAnimation(anim.getFig(), anim.actualize, 
                            frames=len(hist), interval=200, blit=True)
HTML(a.to_html5_video())

Callecting data 5000 iteration
Creating animation ...


### Funkcja sinus

In [9]:
# %matplotlib inline

control_X = np.linspace(0, 2, 21)
control_Y = [float(np.sin(1.5*np.pi*i)) for i in control_X]

X = np.array([[float(i)] for i in np.linspace(0, 2, 101)])
Y = np.array([[float(np.sin(1.5*np.pi*i))] for i in np.linspace(0, 2, 101)])

hist, model = runNetwork(X, Y, 'relu', 'mean_squared_error', 'adam', visible=True)
print('Creating animation ...')
anim = Anim(hist, 0, 2, -2, 2, control_X, control_Y)
a = animation.FuncAnimation(anim.getFig(), anim.actualize, 
                            frames=len(hist), interval=200, blit=True)
HTML(a.to_html5_video())

Callecting data 5000 iteration
Creating animation ...


## Moduł testowy szukający najlepszych wartości dla sieci
(na przykładzie funkcji kwadratowej)

In [6]:
X = np.array([[float(i)] for i in np.linspace(-50, 50, 101)])
Y = np.array([[float(i ** 2)] for i in np.linspace(-50, 50, 101)])

activations = ['relu', 'sigmoid', 'tanh', 'linear', 'softmax']
opts = ['adam', 'adadelta', 'adagrad', 'adamax', 'sgd']

for a in activations:
    for o in opts:
        hist, model = runNetwork(X, Y, a, 'mean_squared_error', o, visible=False)
        print(f'loss: {model.evaluate(X, Y, verbose=0)}\t\t {a}\t\t {o}')

loss: 416.77699566831683	 relu	 adam
loss: 332.3332737082302	 relu	 adadelta
loss: 69852.00108292079	 relu	 adagrad
loss: 14897.767094678218	 relu	 adamax
loss: 580490.5792079208	 relu	 sgd
loss: 595467.5462175123	 sigmoid	 adam
loss: 379919.1202042079	 sigmoid	 adadelta
loss: 1215166.7831064356	 sigmoid	 adagrad
loss: 728839.6031965502	 sigmoid	 adamax
loss: 584343.2698019802	 sigmoid	 sgd
loss: 593317.3578003987	 tanh	 adam
loss: 395282.4540687788	 tanh	 adadelta
loss: 1211193.197091584	 tanh	 adagrad
loss: 718892.1176562546	 tanh	 adamax
loss: 607881.1584158416	 tanh	 sgd
loss: 578029.6658415842	 linear	 adam
loss: 578070.6683168317	 linear	 adadelta
loss: 578383.5148514851	 linear	 adagrad
loss: 578113.5235148515	 linear	 adamax
loss: nan	 linear	 sgd
loss: 1133623.396039604	 softmax	 adam
loss: 1003085.5667543317	 softmax	 adadelta
loss: 1282146.5748762377	 softmax	 adagrad
loss: 1159641.2682549504	 softmax	 adamax
loss: 593106.551980198	 softmax	 sgd


## Moduł testowy szukający najlepszej randomizacji
(na przykładzie sinusoidy)

In [6]:
X = np.array([[float(i)] for i in np.linspace(0, 2, 101)])
Y = np.array([[float(np.sin(1.5*np.pi*i))] for i in np.linspace(0, 2, 101)])

for r in range(20):
    np.random.seed(r)
    tf.random.set_seed(r)

    hist, model = runNetwork(X, Y, 'relu', 'mean_squared_error', 'adam', visible=False)
    print(f'{r}\t loss: {model.evaluate(X, Y, verbose=0)}')

0	 loss: 0.09944433298441444
1	 loss: 0.05973525537122594
2	 loss: 0.004998575268995644
3	 loss: 0.09786751156322437
4	 loss: 0.003174388790410934
5	 loss: 0.1007053942078411
6	 loss: 0.09954953990360298
7	 loss: 0.0006050063783561092
8	 loss: 0.0053221118581103215
9	 loss: 0.004018992630506654
10	 loss: 0.011682798145431102
11	 loss: 0.004826228367164731
12	 loss: 0.09811005553249086
13	 loss: 0.0012450156373774062
14	 loss: 0.3300901964366768
15	 loss: 0.12078656241445258
16	 loss: 0.10097188082071815
17	 loss: 0.0021817572230454717
18	 loss: 0.10986987505532285
19	 loss: 0.0006961224077380087


Jak widać po przeprowadzeniu testów, najlepszą funckją aktywacji jest ReLU. Najbardziej odpowiednio wypada optymalizacja algorytmem Adam. Dobre wyniki otrzymuje się dla seed = 19.